In [0]:
# =========================
# UNION Bronze -> Silver (WANTED only, force STRING before union)
# =========================
from pyspark.sql import functions as F
from functools import reduce

# ===== CONFIG =====
CATALOG    = "lapse_scoring_dev"
SRC_SCHEMA = "01_bronze"
DST_SCHEMA = "02_silver"

TABLES = [
    "chandra_201701_trad_master_bronze",
    "chandra_201701_ul_master_bronze",
    "chandra_201702_trad_master_bronze",
    "chandra_201702_ul_master_bronze",
    "chandra_201703_trad_master_bronze",
    "chandra_201703_ul_master_bronze",
    "chandra_201704_trad_master_bronze",
    "chandra_201704_ul_master_bronze",
    "chandra_201705_trad_master_bronze",
    "chandra_201705_ul_master_bronze",
    "chandra_201706_trad_master_bronze",
    "chandra_201706_ul_master_bronze",
    "chandra_201707_trad_master_bronze",
    "chandra_201707_ul_master_bronze",
    "chandra_201708_trad_master_bronze",
    "chandra_201708_ul_master_bronze",
    "chandra_201709_trad_master_bronze",
    "chandra_201709_ul_master_bronze",
    "chandra_201710_trad_master_bronze",
    "chandra_201710_ul_master_bronze",
    "chandra_201711_trad_master_bronze",
    "chandra_201711_ul_master_bronze",
    "chandra_201712_trad_master_bronze",
    "chandra_201712_ul_master_bronze",
    "chandra_201801_trad_master_bronze",
    "chandra_201801_ul_master_bronze",
    "chandra_201802_trad_master_bronze",
    "chandra_201802_ul_master_bronze",
    "chandra_201803_trad_master_bronze",
    "chandra_201803_ul_master_bronze",
    "chandra_201804_trad_master_bronze",
    "chandra_201804_ul_master_bronze",
    "chandra_201805_trad_master_bronze",
    "chandra_201805_ul_master_bronze",
    "chandra_201806_trad_master_bronze",
    "chandra_201806_ul_master_bronze",
    "chandra_201807_trad_master_bronze",
    "chandra_201807_ul_master_bronze",
    "chandra_201808_trad_master_bronze",
    "chandra_201808_ul_master_bronze",
    "chandra_201809_trad_master_bronze",
    "chandra_201809_ul_master_bronze",
    "chandra_201810_trad_master_bronze",
    "chandra_201810_ul_master_bronze",
    "chandra_201811_trad_master_bronze",
    "chandra_201811_ul_master_bronze",
    "chandra_201812_trad_master_bronze",
    "chandra_201812_ul_master_bronze",
    "chandra_201901_trad_master_bronze",
    "chandra_201901_ul_master_bronze",
    "chandra_201902_trad_master_bronze",
    "chandra_201902_ul_master_bronze",
    "chandra_201903_trad_master_bronze",
    "chandra_201903_ul_master_bronze",
    "chandra_201904_trad_master_bronze",
    "chandra_201904_ul_master_bronze",
    "chandra_201905_trad_master_bronze",
    "chandra_201905_ul_master_bronze",
    "chandra_201906_trad_master_bronze",
    "chandra_201906_ul_master_bronze",
    "chandra_201907_trad_master_bronze",
    "chandra_201907_ul_master_bronze",
    "chandra_201908_trad_master_bronze",
    "chandra_201908_ul_master_bronze",
    "chandra_201909_trad_master_bronze",
    "chandra_201909_ul_master_bronze",
    "chandra_201910_trad_master_bronze",
    "chandra_201910_ul_master_bronze",
    "chandra_201911_trad_master_bronze",
    "chandra_201911_ul_master_bronze",
    "chandra_201912_trad_master_bronze",
    "chandra_201912_ul_master_bronze",
]

WANTED = [
    "chdrnum","shortdesc","aracde","cnttype","crtable","hissdte","subdat",
    "payfreq","sumins","statcode","lifenum","age02","dob02","gender",
    "smokeind","annprem01","ptdate","btdate","clntnum",
    "age01","dob01","pldesc01","pldesc02"
]

TARGET_TABLE = "chandra_union_silver"
WRITE_MODE   = "overwrite"  # gunakan overwrite supaya overwriteSchema bekerja

# ===== Helpers =====
def select_with_missing_as_null(df, wanted, default_type="string"):
    """Select kolom sesuai urutan WANTED; kolom yg hilang diisi NULL (cast default_type)."""
    cols = df.columns
    sel  = [F.col(c) if c in cols else F.lit(None).cast(default_type).alias(c) for c in wanted]
    return df.select(*sel)

def force_all_string(df, cols):
    """Paksa semua kolom di 'cols' menjadi STRING (hindari implicit cast saat union)."""
    for c in cols:
        if c in df.columns:
            df = df.withColumn(c, F.col(c).cast("string"))
    return df

def union_all(dfs):
    return reduce(lambda a,b: a.unionByName(b, allowMissingColumns=True), dfs)

# ===== Ensure destination schema exists =====
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.`{DST_SCHEMA}`")

# ===== Load, align (NULL-fill + STRING), and union =====
dfs = []
for t in TABLES:
    src_fqn = f"{CATALOG}.`{SRC_SCHEMA}`.{t}"
    df0 = spark.table(src_fqn)

    # 1) Ambil WANTED dengan kolom hilang -> NULL
    dfi = select_with_missing_as_null(df0, WANTED, default_type="string")

    # 2) 🔑 Paksa SEMUA kolom WANTED menjadi STRING sebelum union (penting!)
    dfi = force_all_string(dfi, WANTED)

    # 3) Lineage
    dfi = dfi.withColumn("_source_table", F.lit(t)) \
             .withColumn("_ingest_ts_union", F.current_timestamp())

    dfs.append(dfi)

# Union (sekarang aman karena tipe sudah seragam STRING)
df_union = dfs[0] if len(dfs) == 1 else union_all(dfs)

# ===== Write to Silver (schema STRING, overwrite) =====
TARGET_FQN = f"{CATALOG}.`{DST_SCHEMA}`.{TARGET_TABLE}"

(df_union.write
    .mode(WRITE_MODE)                 # pakai overwrite agar schema di-redefine
    .option("mergeSchema", "true")
    .option("overwriteSchema", "true")  # pastikan schema target mengikuti df_union (semua STRING)
    .format("delta")
    .saveAsTable(TARGET_FQN))

print(f"Write OK -> {TARGET_FQN}")
print("Rows:", spark.table(TARGET_FQN).count())
display(spark.sql(f"SELECT * FROM {TARGET_FQN} LIMIT 50"))

In [0]:
# =========================
# Rename columns to business names (make a new Silver table)
# =========================
from pyspark.sql import functions as F

CATALOG     = "lapse_scoring_dev"
SRC_SCHEMA  = "02_silver"
SRC_TABLE   = "chandra_union_silver"          # tabel hasil union (semua STRING)
DST_SCHEMA  = "02_silver"
DST_TABLE   = "chandra_union_bn_silver"       # bn = business names

# Mapping: old -> new (edit jika perlu)
BUSINESS_MAP = {
    "chdrnum":   "Contract_Number",
    "shortdesc": "Product_Category",
    "aracde":    "Product_Group",
    "cnttype":   "Product",
    "crtable":   "Product_Detail",
    "hissdte":   "Policy_Issue_Date",
    "subdat":    "Submission_Date",
    "payfreq":   "Payment_Frequency",
    "sumins":    "Sum_Assured",
    "statcode":  "Policy_Status_Code",
    "lifenum":   "Life_Number",
    "age02":     "Age_of_the_Life_Insured",
    "dob02":     "Date_of_Birth_of_the_Life_Insured",
    "gender":    "Gender_of_the_Life_Insured",
    "smokeind":  "Smoking_Indicator",
    "annprem01": "Annualized_Premium",
    "ptdate":    "Paid_to_Date",
    "btdate":    "Billing_to_Date",
    "clntnum":   "Client_Number",
    "age01":     "Age_of_the_Policy_Owner",
    "dob01":     "Date_of_Birth_of_the_Policy_Owner",
    "pldesc01":  "Name_of_the_Policy_Owner",
    "pldesc02":  "Name_of_the_Life_Insured",
    # lineage
    "_source_table":     "source_table",
    "_ingest_ts_union":  "union_ingest_ts",
}

SRC_FQN = f"{CATALOG}.`{SRC_SCHEMA}`.{SRC_TABLE}"
DST_FQN = f"{CATALOG}.`{DST_SCHEMA}`.{DST_TABLE}"

df = spark.table(SRC_FQN)

# cek kolom yang ada vs mapping
existing = set(df.columns)
mapped_existing = {k:v for k,v in BUSINESS_MAP.items() if k in existing}
missing_in_source = [k for k in BUSINESS_MAP.keys() if k not in existing]
if missing_in_source:
    print("⚠️ Missing in source (ignored):", missing_in_source)

# select dengan alias business
df_bn = df.select([F.col(k).alias(v) for k,v in mapped_existing.items()])

# tulis sebagai tabel baru
(df_bn.write
    .mode("overwrite")
    .option("mergeSchema","true")
    .option("overwriteSchema","true")
    .format("delta")
    .saveAsTable(DST_FQN))

print(f"Write OK -> {DST_FQN}")
display(spark.sql(f"SELECT * FROM {DST_FQN} LIMIT 50"))


In [0]:
# ============================================
# CAST tipe data dari BN -> simpan ke tabel BARU
# ============================================
from pyspark.sql import functions as F

# --- CONFIG ---
CATALOG      = "lapse_scoring_dev"
SRC_SCHEMA   = "02_silver"
SRC_TABLE    = "chandra_union_bn_silver"        # tabel BN sumber (string-heavy)
DST_SCHEMA   = "02_silver"
DST_TABLE    = "chandra_union_dt_silver"  # >>> tabel BARU hasil casting

SRC_FQN = f"{CATALOG}.`{SRC_SCHEMA}`.{SRC_TABLE}"
DST_FQN = f"{CATALOG}.`{DST_SCHEMA}`.{DST_TABLE}"

# Pastikan schema tujuan ada
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.`{DST_SCHEMA}`")

# --- LOAD ---
df = spark.table(SRC_FQN)

# 1) Trim semua kolom string
for c, t in df.dtypes:
    if t.startswith("string"):
        df = df.withColumn(c, F.trim(F.col(c)))

# 2) Uppercase untuk kode/flag (sesuaikan jika ada tambahan)
uppercase_cols = [
    "Policy_Status_Code",
    "Payment_Frequency",
    "Gender_of_the_Life_Insured",
    "Smoking_Indicator",
]
for c in uppercase_cols:
    if c in df.columns:
        df = df.withColumn(c, F.upper(F.col(c)))

# 3) Parse TANGGAL: seluruh data berformat yyyyMMdd (tolerant → NULL jika gagal)
date_cols = [
    "Policy_Issue_Date",
    "Submission_Date",
    "Paid_to_Date",
    "Billing_to_Date",
    "Date_of_Birth_of_the_Policy_Owner",
    "Date_of_Birth_of_the_Life_Insured",
]
for c in date_cols:
    if c in df.columns:
        df = df.withColumn(
            c,
            F.expr(f"try_to_date(regexp_replace(CAST({c} AS STRING), '[^0-9]', ''), 'yyyyMMdd')")
        )

# 4) Numerik (double): bersihkan locale lalu cast ke double
def to_double(col):
    s = F.regexp_replace(col.cast("string"), r"\s", "")
    return (
        F.when(s.rlike(r"^\d{1,3}(\.\d{3})+,\d+$"), F.regexp_replace(F.regexp_replace(s, r"\.", ""), ",", "."))
         .when(s.rlike(r"^\d+,\d+$"),               F.regexp_replace(s, ",", "."))
         .when(s.rlike(r"^\d{1,3}(,\d{3})+\.\d+$"), F.regexp_replace(s, ",", ""))
         .otherwise(s)
    ).cast("double")

double_cols = [
    "Sum_Assured",
    "Annualized_Premium",
    # Tambahkan kolom numeric lain jika ada, mis.:
    # "Loan_Value", "Total_Fee", "Coverage_Debt"
]
for c in double_cols:
    if c in df.columns:
        df = df.withColumn(c, to_double(F.col(c)))

# 5) Integer: usia (Life_Number dibiarkan string jika itu ID)
int_cols = [
    "Age_of_the_Policy_Owner",
    "Age_of_the_Life_Insured",
]
for c in int_cols:
    if c in df.columns:
        df = df.withColumn(c, F.col(c).cast("int"))

# 6) Pastikan ID/teks utama tetap STRING (defensif)
id_string_cols = [
    "Contract_Number", "Client_Number", "Life_Number",
    "Product_Category", "Product_Group", "Product", "Product_Detail",
    "Name_of_the_Policy_Owner", "Name_of_the_Life_Insured",
    "source_table"
]
for c in id_string_cols:
    if c in df.columns:
        df = df.withColumn(c, F.col(c).cast("string"))

# 7) Tulis ke TABEL BARU (tidak menyentuh tabel sumber)
(df.write
    .mode("overwrite")              # buat/replace tabel baru
    .option("mergeSchema", "true")
    .option("overwriteSchema", "true")
    .format("delta")
    .saveAsTable(DST_FQN))

print(f"Typed table written -> {DST_FQN}")
print("Rows:", spark.table(DST_FQN).count())
display(spark.sql(f"DESCRIBE {DST_FQN}"))
display(spark.sql(f"SELECT * FROM {DST_FQN} LIMIT 20"))
